# 爬取 ProgrammableWeb

In [ ]:
import requests
from bs4 import BeautifulSoup
import threading

mashup_list_base = 'https://www.programmableweb.com/category/all/mashups?deadpool=1&order=created&sort=desc&page='
base_url = 'https://www.programmableweb.com'
page = 0  # 当前页数
max_page = 319  # 指定最大页数

In [ ]:
# 从文件读取hrefs，就不用再爬一遍了
href_file_name = 'hrefs-0905-2.txt'
def read_hrefs(filename):
    hrefs_date = {}
    with open(filename, mode='r', encoding='utf-8') as href_file:
        for line in href_file:
            split = line.split('#####')
            hrefs_date[split[0]] = split[1]
            # hrefs.append(line)
    return hrefs_date
actived_hrefs = read_hrefs(href_file_name)
hrefs = list(hrefs_date.keys())
hrefs[:20]

['https://www.programmableweb.com/mashup/petdoption',
 'https://www.programmableweb.com/mashup/landedcostio-consolidated-screening-list',
 'https://www.programmableweb.com/mashup/voice-apps',
 'https://www.programmableweb.com/mashup/best-gaming-pc-deals',
 'https://www.programmableweb.com/mashup/keyword-research-tool',
 'https://www.programmableweb.com/mashup/standuply',
 'https://www.programmableweb.com/mashup/loanprofy',
 'https://www.programmableweb.com/mashup/datantify',
 'https://www.programmableweb.com/mashup/penny-parrot',
 'https://www.programmableweb.com/mashup/banksnearme',
 'https://www.programmableweb.com/mashup/hashtag-suggestions',
 'https://www.programmableweb.com/mashup/digireality',
 'https://www.programmableweb.com/mashup/autochat-whatsapp-business-api-sandbox',
 'https://www.programmableweb.com/mashup/refinansierenet',
 'https://www.programmableweb.com/mashup/mars-explorer',
 'https://www.programmableweb.com/mashup/bitcoin-digital-downloads-and-terminal-2018',
 'http

In [ ]:
hrefs = []  # 详情页列表

hrefs_date = {}

# 多线程获取详情页的链接
def get_detail_links(thread_count, page):
    thread_count[0] += 1

    mashup_list_page = mashup_list_base + str(page)  # mashup列表页面的URL
    req = requests.get(mashup_list_page)
    html_text = req.text

    bs1 = BeautifulSoup(html_text)
    div = bs1.find('div', class_='view-content')
    table = div.find('table')
    tbody = table.find('tbody')
    trs = tbody.find_all('tr')  # 所有tr标签，每一个元素就是一个列表行，后续进一步获取
    for tr in trs:  # 遍历每行，获取详情页的链接
        td_title = tr.find('td', class_='views-field-title')
        td_date = tr.find('td', class_='views-field-created')
        a = td_title.find('a')
        href1 = a.get('href')
        href = base_url + href1  # detail_page
        date = td_date.text.strip()

        hrefs_date[href] = date

        # hrefs.append(href + '#####' + date)  # 加入这个详情页链接到hrefs列表中，方便后续的一次性访问

    thread_count[0] -= 1

In [ ]:
max_thread = 40
thread_count = [0]

page_index = 0
while True:
    if page_index >= max_page:
        break
    else:
        if thread_count[0] < max_thread:
            # 创建新新线程
            t = threading.Thread(target=get_detail_links, args=(thread_count, page_index,))
            t.start()
            print('正在获取页码', page_index)
            page_index += 1


In [ ]:
len(hrefs)

In [ ]:
# 保存所有链接到文件
with open('hrefs-0910-1.txt', mode='w', encoding='utf-8') as href_file:
    for href in hrefs_date.keys():
        href_file.write(href + '#####' + hrefs_date[href] + '\n')

In [ ]:
# 现在hrefs_date 是包括了Deadpool的mashup
# 和旧的mashup 进行对比然后挑出来没有爬取过的mashup

actived_list = list(actived_hrefs.keys())
all_list = list(hrefs_date.keys())

# deactivated
result4 = list(set(all_list).difference(set(actived_list)))
len(result4)

1530

In [ ]:
# %% 分割列表的辅助函数
def list_split(list_info, per_list_len):
    """
    :param list_info:   列表
    :param per_list_len:  每个小列表的长度
    :return:
    """
    list_of_group = zip(*(iter(list_info),) * per_list_len)
    end_list = [list(i) for i in list_of_group]
    count = len(list_info) % per_list_len
    end_list.append(list_info[-count:]) if count != 0 else end_list
    return end_list

In [ ]:
# 9.10
hrefs = all_list

1530


In [ ]:
# 访问hrefs中的每一个href，最好多线程访问，不然太慢了
max_thread = 40
thread_count = [0]
hrefs_split = list_split(hrefs, 8)
mashups = []
failed_hrefs = []

In [ ]:
len(hrefs_split)

192

In [ ]:
import json
from data.crawler.data_objects import Api, ApiVersion, Mashup, DeadpoolMashup



# 2020.09.04 New requirements
- 爬取与Mashup的相关的API的一些信息，不仅仅是API的名字
- 创建一个字典保存API的URL->API对象的映射。
- 下次发现使用了相同的API的时候就不需要额外爬取一遍了
- 与API爬虫有重复爬取的部分

In [ ]:
apis = {}
failed_api_hrefs = []
def get_api(url):
    if url in apis:
        print('请求的API ' + url + ' 直接返回')
        return apis[url]
    else:
        print('请求的API ' + url + ' 正在获取')
        # 单线程爬吧，每个mashup相关的api数量也不是特别多
        req_api = requests.get('https://www.programmableweb.com' + url)
        bs_api = BeautifulSoup(req_api.text, features='html.parser')
        # 找node-header，锁定标题
        node_header = bs_api.find('div', class_='node-header')
        if node_header is None:
            # 失败
            failed_api_hrefs.append(url)
            print('failed')
            return None
        title = node_header.find('h1').text.strip()     # title

        tags = []   #tags
        div_tags = bs_api.find('div', class_='tags')
        tags_a = div_tags.find_all('a')
        for a in tags_a:
            tags.append(a.text.strip())

        intro = bs_api.find('div', class_='intro')
        description = ''
        if intro is None:
            # 失败
            failed_api_hrefs.append(url)
            print('not found intro')
        else:
            description_div = intro.find('div', class_='api_description')
            if description_div is None:
                failed_api_hrefs.append(url)
                print('not found description_div')
            else:
                description = description_div.text.strip()  #description

        # 定位版本versions
        versions = []
        version_field = bs_api.find('div', id='version-details-field')
        version_results = version_field.find_all('div', class_='version-result-set')

        # 开始获取版本
        for result in version_results:
            divs = result.find_all('div')
            version_title = ''
            version_style = ''
            version_status = ''
            version_version = ''
            version_submit_date = ''
            for div in divs:
                # 直接找所有div，根据label去匹配
                label = div.find('label')
                if label is None:
                    # 不是我要的div
                    continue
                else:
                    lt = label.text.strip()
                    version_info = div.text.strip()
                    if lt == 'Title:':
                        version_title = version_info.split(':')[1]
                    elif lt == 'Style:':
                        version_style = version_info.split(':')[1]
                    elif lt == 'Version:':
                        version_version = version_info.split(':')[1]
                    elif lt == 'Status:':
                        # 这个实在没发现什么好的办法，只能碰碰运气了
                        spans = div.find_all('span')
                        # 因为0是那个mobile，我不需要，只需要1
                        span = spans[1]
                        version_status = span.text.strip()
                    elif lt == 'Submitted:':
                        version_submit_date = version_info.split(':')[1]
            version = ApiVersion(version_title, version_style, version_version, version_status, version_submit_date)
            versions.append(version)

        # 创建API对象
        api = Api(title, tags, description, url, versions)
        apis[url] = api
        print(api.title)
        print('请求的API ' + url + ' 获取完成！')
        return api


In [ ]:
# 下面的函数最后修改于2020.09.04 22:49
def get_mashup(url, is_dead_mashup=False):
    # -------------
        # 逐步获取关键数据
        req1 = requests.get(href)   # 网络请求
        bs2 = BeautifulSoup(req1.text, features='html.parser')  # soup 解析html
        
        # 获取标题
        node_header = bs2.find('div', class_='node-header')
        if node_header is None:
            print('href "' + href + '" does not contain class=node-header.')
            failed_hrefs.append(href)
            return None
        title = node_header.find('h1').text   #标题

        # 获取tags
        tags = []
        # tags_div = node_header.find('div', class_='tags')
        # tags_a = tags_div.find_all('a')
        # for a_tag in tags_a:
        #     # a标签
        #     tags.append(a_tag.text.strip())

        # 获取description
        intro = bs2.find('div', class_='intro')
        description_div = intro.find('div', class_='field-item')
        description = ''
        if description_div is None:
            # 没有找到description的div，也是爬取失败了
            print(url + ' not found class=field-item (description_div)')
        else:
            description = description_div.text  # description
        
        # 获取Spec
        tabs_content = bs2.find('div', id='tabs-content')
        fields = tabs_content.find_all('div', class_='field')
        # 获取完成
        # --------------
        related_apis = []
        categories = []
        url = 'Undefined'
        mashup_type = 'Undefined'
        for field in fields:
            label = field.find('label').text.strip()  
            # 通过label判断而不是按照顺序，防止某些页面的顺序不一致而出现问题
            if label == 'Related APIs':
                # related apis
                related_apis = [get_api(a0.get('href')) for a0 in field.find_all('a')]
            elif label == 'Categories':
                # 分类
                categories = [a0.text for a0 in field.find_all('a')]
            elif label == 'URL':
                # 获取URL
                url_a = field.find('a')
                if url_a is None:
                    # URL 为空
                    print(title + "'s url is Undefined")
                else:
                    url = url_a.get('href')
            elif label == 'Mashup/App Type':
                # 获取type
                mashup_type_span = field.find('span')
                if mashup_type_span is None:
                    # type 为空
                    print(title + "'s type is Undefined")
                else:
                    mashup_type = mashup_type_span.text.strip()

        # 判断一下是不是deadpooled的mashup
        if is_dead_mashup:
            # 获取changelog
            changelogs = get_changelogs(url)
            deprecated_date = 'Not sure'
            if len(changelogs) >= 1:
                deprecated_date = changelogs[0].split('|')[0]
            mashup = DeadpoolMashup(title, tags, description, related_apis, categories, url, mashup_type, changelogs, deprecated_date, hrefs_date[href])
        else:
            # 创建mashup对象
            mashup = Mashup(title, tags, description, related_apis, categories, url, mashup_type, hrefs_date[href])
        return mashup


def get_detail_info(thread_count, hrefs):
    """
    遍历传入的hrefs列表，访问它的每一个href，
    然后获取这个页面中的 标题 title、描述 description、相关API related_apis
    并且创建Mashup对象，加入mashups的列表中。
    :param thread_count: 当前已启用的线程数量
    :param hrefs: 链接列表，每个元素都是一个链接，在这个函数中会被遍历，发起网络请求，爬取返回的html中的内容
    :return: none
    """
    thread_count[0] += 1    # 已启用线程数加1
    for href in hrefs:
        mashup = get_mashup(href)
        mashups.append(mashup)
    thread_count[0] -= 1    # 已启用线程数减1

In [ ]:
def get_changelogs(url):
    changelogs = []
    change_req = requests.get(url + '/changelog')
    cbs = BeautifulSoup(change_req.text, features='html.parser')    # changelog beautiful soup
    changelog_divs = cbs.find_all('p', class_='changelog_divider')
    for changelog_div in changelog_divs:
        changelog_text = changelog_div.get_text("|", strip=True)
        if 'deadpool' in changelog_text.lower():
            print(True)
            # 有用的changelog
            changelogs.append(changelog_text)
    return changelogs

In [ ]:
failed_hrefs = []

In [ ]:
mashups = []

In [ ]:
# 开启多线程访问hrefs
i = 0
length = len(hrefs_split[:1])
while True:
    if i >= length:
        break
    if thread_count[0] < max_thread:
        if i % 100 == 0:
            print('i =', i)
        # 创建新新线程
        t = threading.Thread(target=get_detail_info, args=(thread_count, hrefs_split[i]))
        t.start()
        print('已创建新线程。当前处理第', (i + 1), '个子列表，总共有', length, '个子列表')
        i += 1

i = 0
已创建新线程。当前处理第 1 个子列表，总共有 1 个子列表


In [ ]:
# %% 写文件
# 将mashups写入文件，作为数据集
file = open('data0906-1-temp.txt', mode='w', encoding='utf-8')
file.write('Dataset Size(Mashup Count): ' + str(len(mashups)) + '\n')

for mashup in mashups:

    bbb = mashup.json()

    # print(bbb)
    file.write(bbb)
file.flush()
file.close()

请求完成: https://www.programmableweb.com/mashup/smarkets
related api
请求的API /api/amazon-product-advertising 直接返回
categories
url
https://www.programmableweb.com/mashup/smarkets 完成
正在获取: https://www.programmableweb.com/mashup/pumps.ie


In [ ]:
# 写文件
with open('data0906-1.txt', mode='w', encoding='utf-8') as file:
    file.write(json.dumps(obj=mashups, default=lambda x : x.__dict__, sort_keys=False, indent=2))

# Mashups accessibility

In [ ]:
def dict2mashup(mashup_dict, is_deadpool_mashup=False):
    # 转为dict
    title = mashup_dict['title']
    descrip = mashup_dict['description']
    categories = mashup_dict['categories']

In [ ]:
mashup_file = 'data0906-1.txt'
deadpool_mashup_file = 'deadpool_mashups-0910-1.txt'
mashups1 = {}
deadpool_mashups = {}
with open(deadpool_mashup_file, 'r', encoding='utf-8') as file:
    text = file.read()
    print(len(text))
    mashup_dict_list = json.loads(text)
    print(len(mashup_dict_list))
    for m_dict in mashup_dict_list:
        if m_dict is not None:
            mashups1[m_dict['url']] = m_dict
print(len(mashups1))

In [ ]:
# 定义一个类存储东西
class MashupVisit:
    def __init__(self, title, url, status_code, is_accessible):
        self.title = title
        self.url = url
        self.status_code = status_code
        self.is_accessible = is_accessible

    def json(self):
        return json.dumps(obj=self, default=lambda x : x.__dict__, sort_keys=False, indent=2)


In [ ]:
# 尝试访问所有URL，记录访问状态
visits = []
for url in list(mashups1.keys())[-10:-1]:
    print(url)
    try:
        req = requests.get(url)
        status_code = req.status_code
    except:
        status_code = -1
    is_accessible = False
    mashup_title = mashups1[url]['title']
    if status_code == 200 or status_code == 429:
        bs = BeautifulSoup(req.text, features='html.parser')
        # 先判断这个网页中是否包含"mashup"字样，如果包含，就进一步判断标题
        html_text = bs.text
        if 'mashup' in html_text.lower():
            title = bs.find('head').find('title')
            if title is not None:
                if 'for sale' not in title.text.lower():
                    # For Sale 很可能说明这个域名已经跪了，都正在出售了
                    # False
                    is_accessible = True
            else:
                # 标题为空，暂时标记为True吧
                is_accessible = True
        else:
            # mashup 没有被包含在整个网页的内容中
            is_accessible = False

    visit = MashupVisit(mashup_title, url, status_code, is_accessible)
    visits.append(visit)

In [ ]:
def visit_mashup(url):
    try:
        req = requests.get(url)
        status_code = req.status_code
    except:
        status_code = -1
    is_accessible = False
    mashup_title = mashups1[url]['title']
    if status_code == 200 or status_code == 429:
        bs = BeautifulSoup(req.text, features='html.parser')
        # 不可访问或报错直接False
        # 如果可访问，先判断网页HTML代码中中是否包含这个Mashup的名字
        # 如果包含名字，进一步判断是否for sale，for sale 就False
        # 如果不包含名字，就把名字拆成单词看在不在网页中出现，有一个单词出现就标为True
        # 一个单词都不包含的话，就判断内容是否包含 mashup 字眼，有就True
        # 如果还不包含mashup字眼，就标记为False
        html_text = req.text
        html_text_lower = html_text.lower()
        if mashup_title.replace('Mashup:', '').strip().lower() in html_text_lower:
            # 此时可以直接标为True
            is_accessible = True
            # 包含名字

            # 判断 for sale 与否
            head = bs.find('head')
            if head is not None:    # 如果没有标题，就不判断啊是否for sale 了
                title = head.find('title')
                if title is not None:
                    # 含有 title 字段
                    if 'for sale' in title.text.lower():
                        # For Sale 很可能说明这个域名已经跪了，都正在出售了
                        # False
                        is_accessible = False
        else:
            # 把标题按照空格拆分，判断是否至少包含一个单词
            page_lower = bs.text.lower()
            title_splits = mashup_title.lower().split(' ')
            exists = False
            for element in title_splits:
                if element in page_lower:
                    exists = True
                    break
            if exists:
                is_accessible = True
            elif 'mashup' in html_text_lower:   # 不包含名字，检查是否包含mashup 字眼
                # 包含
                is_accessible = True
            else:
                # 不包含
                is_accessible = False

    visit = MashupVisit(mashup_title, url, status_code, is_accessible)
    return visit

def threads_visit_mashups(thread_count, mashup_links):
    thread_count[0] += 1
    for url in mashup_links:
        visits.append(visit_mashup(url))
    thread_count[0] -= 1

urls = list(mashups1.keys())

In [ ]:
def list_split(list_info, per_list_len):
    list_of_group = zip(*(iter(list_info),) * per_list_len)
    end_list = [list(i) for i in list_of_group]  # i is a tuple
    count = len(list_info) % per_list_len
    end_list.append(list_info[-count:]) if count != 0 else end_list
    return end_list

In [ ]:
max_thread = 100
thread_count = [0]
now_list = list_split(urls, 8)
visits = []
i = 0
length = len(now_list)
while True:
    if i >= length:
        break
    if thread_count[0] < max_thread:
        # 创建新新线程
        t = threading.Thread(target=threads_visit_mashups, args=(thread_count, now_list[i]))
        t.start()
        print('已创建新线程。当前处理第', (i + 1), '个子列表，总共有', length, '个子列表')
        i += 1


In [ ]:
with open('mashup_accessibility.txt', 'w', encoding='utf-8') as file:
    file.write(json.dumps(obj=visits, default=lambda x : x.__dict__, sort_keys=False, indent=2))

